In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.900369449497976, 'Fn': 1.184383282690447, 'kn': 0.6414529928884467, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 1.0415462755795253, 'Fn': 1.7637226511189565, 'kn': 0.6348125738082766, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.932903940019097, 'Fn': 2.1321345406869066, 'kn': 0.24902581391857953, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.8278081871246422, 'Fn': 3.8199302863134226, 'kn': 0.35174154718363215, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 1.0432905838092195, 'Fn': 3.913825949541608, 'kn': 0.43740643005316393, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.9766658416311175, 'Fn': 2.4920855793255217, 'kn': 0.5668848317824697, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.833279172556006, 'Fn': 1.5048957788256199, 'kn': 0.3583132239990442, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.9124006

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5
 2.5 2.5]
Uniform p: [0.87061796 0.3457042  0.31828236 3.2508949  2.41399649 0.77871702
 1.20114963 1.08456425 1.79500847 2.09623896 4.41027627 1.47982962
 1.49705332 0.77798712 1.52674967 0.28983803 0.73579232 4.12350645
 2.53479259 2.56085389]
Uniform Dmax: 0.9024129987955131
Uniform r: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
social welfare: -46.48253744367689
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.1434783369486628', 'MD 2 Dn: 0.5588148096297852', 'MD 3 Dn: 0.6912362657793774', 'MD 4 Dn: 0.03705444983432793', 'MD 5 Dn: 0.04871537232012768', 'MD 6 Dn: 0.16414754128416828', 'MD 7 Dn: 0.09393307987232885', 'MD 8 Dn: 0.10041486376099848', 'MD 9 Dn: 0.062229402648298876', 'MD 10 Dn: 0.05965829956568769', 'MD 11 Dn: 0.029734645282242546', 'MD 12 Dn: 0.07275945228980389', 'MD 13 Dn: 0.07084992481921967', 'MD 14 Dn: 0.1506954341845181', 'MD 15 Dn: 0.07210648752419745', 'MD 16 Dn: 0.9

In [4]:
T = 100
N = len(MDs)
lam_pro, p_pro, r_pro, Dmax_pro = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.proportional_baseline(esp,MDs,43)
    lam_pro += l
    p_pro += p 
    r_pro += r
    Dmax_pro += d
lam_pro, p_pro, r_pro, Dmax_pro = lam_pro/T, p_pro/T, r_pro/T, Dmax_pro/T
print(f"Proportional λ: {lam_pro}")
print(f"Proportional p: {p_pro}")  
print(f"Proportional Dmax: {Dmax_pro}")
print(f"Proportional r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Proportional λ: [0.97813185 1.45658363 1.76083936 3.15471819 3.23226263 2.05810764
 1.24282951 2.74919597 3.20928448 3.55542061 3.80177886 1.34214099
 3.83695585 1.70564953 2.67006073 2.02903671 1.44837616 3.56628358
 2.45255766 3.74978607]
Proportional p: [0.82828559 0.26588496 0.26033392 3.25931654 2.43818207 0.75130928
 1.17618648 1.09924305 1.82792052 2.14259038 4.41492992 1.46981661
 1.57420819 0.73213939 1.53461329 0.2523146  0.67765214 4.12762196
 2.53417852 2.60733311]
Proportional Dmax: 0.9008927227311959
Proportional r: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
social welfare: -46.30057860380447
sum lambda: 49.99999999999999  lambda0: 50
['MD 1 Dn: 0.13084706305953955', 'MD 2 Dn: 0.5453820377624786', 'MD 3 Dn: 0.6845308810663417', 'MD 4 Dn: 0.037437763663346335', 'MD 5 Dn: 0.04928558494405769', 'MD 6 Dn: 0.16227575430424243', 'MD 7 Dn: 0.08832190170799539', 'MD 8 Dn: 0.10095555354041635', 'MD 9 Dn: 0.06299014668659673', 'MD 10 Dn: 0.06052294270468994', 'MD

In [5]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

Non-cooperative λ: [0.77260802 1.084822   3.97698444 4.80775286 3.07090964 2.13084625
 3.36065434 0.84728255 3.87612971 1.04394456 2.58084448 1.36089985
 0.95548016 3.27859234 3.34645679 1.82610006 2.59226851 2.06310222
 2.85357201 4.17074918]
Non-cooperative p: [0.1790983  0.20168657 0.43389783 0.49969571 0.36103346 0.28804975
 0.38444948 0.18271872 0.42536796 0.19968095 0.32152692 0.22385295
 0.19124435 0.37759945 0.38295102 0.26134088 0.3221687  0.28076413
 0.34487663 0.45108464]
Non-cooperative Dmax: 0.7121227027738769
Non-cooperative r: [0.16459499 0.18535407 0.39876096 0.45923054 0.33179711 0.26472359
 0.35331691 0.16792223 0.39092183 0.18351086 0.29548979 0.20572543
 0.17575745 0.34702159 0.3519398  0.2401776  0.29607961 0.25802796
 0.31694866 0.41455599]
social welfare: 78.94145455747358
sum lambda: 50.00000000000001  lambda0: 50
['MD 1 Dn: 0.7062574221822031', 'MD 2 Dn: 0.7060099078364163', 'MD 3 Dn: 0.706098518668646', 'MD 4 Dn: 0.706028533806348', 'MD 5 Dn: 0.706058762130286

In [6]:
lam_con, p_con, r_con, Dmax_con = baseline.contract_baseline(esp,MDs)
print(f"Contract λ: {lam_con}")
print(f"Contract p: {p_con}")  
print(f"Contract Dmax: {Dmax_con}")
print(f"Contract r: {r_con}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_con, p_con))
print("sum lambda:", np.sum(lam_con), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_con[i],lam_con[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_con, " D0:", esp.D0)
max_Dn = max(md.delay(p_con[i], lam_con[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_con == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_con[i], lam_con[i]) <= Dmax_con for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_con <= esp.D0)
print("pn<=Fn:", all(p_con[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_con[i] <= p_con[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_con), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_con[i] >= md.Ln(p_con[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_con[i]) for i,md in enumerate(MDs)]))

求解失败：2 : `xtol` termination condition is satisfied.
Contract λ: [2.45444525 2.51112649 2.48314881 2.50171007 2.5188182  2.5129991
 2.47050732 2.51231431 2.50343515 2.51716453 2.51418773 2.49411939
 2.5211355  2.48192495 2.50269191 2.49976346 2.4814425  2.50484248
 2.5084038  2.50581903]
Contract p: [0.53984995 0.65670676 0.92537766 1.79780526 1.77445195 0.96646135
 0.59683252 1.50489582 1.76372265 1.97535887 2.14785404 0.63858048
 2.13221793 0.87805856 1.52456007 0.99049041 0.69409702 1.96793911
 1.3726667  2.07109635]
Contract Dmax: 0.14487591648077644
Contract r: [1.21130631 1.47938188 1.58546495 3.71192197 4.43576669 2.00725798
 1.06840279 3.73934776 3.97170971 5.51861317 6.14575523 1.36266189
 7.13082778 1.58951855 3.33896826 2.05943658 1.30914055 5.10026561
 2.94792798 5.24935459]
social welfare: 41.824657393329154
sum lambda: 49.999999996447805  lambda0: 50
['MD 1 Dn: 0.25708587773083375', 'MD 2 Dn: 0.18526636479994146', 'MD 3 Dn: 0.12497138008061112', 'MD 4 Dn: 0.060905602667206

In [8]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

SWM λ: [0.78311073 0.96306567 2.52486452 2.85520743 2.07851624 1.53370166
 2.18821351 0.86753167 2.46867776 0.96351821 1.80494354 1.1356518
 0.87738886 2.15523414 2.20989988 1.38655276 1.79175739 1.50523346
 1.93449451 2.63466268 1.9261743  1.46865043 2.04002354 2.8318624
 1.15983798 1.10118331 0.66583453 0.72560107 1.67899439 1.73961164]
SWM p: [0.23950535 0.24843502 0.37528402 0.40020215 0.33871445 0.30022978
 0.34802834 0.24087226 0.36973852 0.25135062 0.31614449 0.26242347
 0.24136615 0.34474136 0.34893837 0.28311238 0.3144733  0.29366079
 0.33000486 0.38794205 0.32667731 0.28906908 0.33522372 0.39971345
 0.26640576 0.26020285 0.2276099  0.23745252 0.30664001 0.31126352]
SWM Dmax: 0.4765305235523106
SWM r: [2.96245693 2.95643887 2.94865644 2.95419055 2.95064023 2.96624613
 2.96548323 2.96529538 2.95197148 2.9689793  2.96001325 2.96603679
 2.95207385 2.96235566 2.95443813 2.9723819  2.97180886 2.97518061
 2.96121437 2.93665617 2.95095792 2.96871791 2.94319473 2.95390526
 2.94565308 

In [ ]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

In [9]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

ADMM λ: [1.07010069 1.18525609 2.24524765 2.44689047 1.95044944 1.57944056
 2.01637771 1.13693844 2.20613277 1.20139455 1.76317761 1.31185314
 1.12170213 1.9959694  2.03596841 1.48329738 1.75254244 1.56283889
 1.850347   2.32511683 1.84750031 1.53676591 1.92741677 2.43310705
 1.31029405 1.2808585  1.01366635 1.01131876 1.67780695 1.72022373]
ADMM p: [0.26240504 0.26615601 0.35285898 0.36748191 0.32841426 0.30383099
 0.33422617 0.2623706  0.34867992 0.27032458 0.31274894 0.27646552
 0.26085687 0.3319453  0.33496909 0.29079753 0.31128216 0.29821412
 0.32321649 0.36312014 0.32032854 0.29446426 0.32616063 0.36775725
 0.27838647 0.27452229 0.25537964 0.26024515 0.30649042 0.30965805]
ADMM Dmax: 0.4766778672211501
ADMM r: [2.98630727 2.9740687  2.92822852 2.92667225 2.93971808 2.96826971
 2.95159493 2.98728536 2.93250817 2.98792591 2.95522643 2.97929304
 2.97188981 2.94930451 2.9404558  2.97863427 2.96720946 2.97816867
 2.95333314 2.914648   2.943481   2.9725772  2.93334735 2.92688582
 2.956

In [9]:
# print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("Nash Product Contract:",model.nash_product_log(esp, MDs, lam_con, p_con, r_con))
print("=============================")
# print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("SW Contract:",model.social_welfare(esp, MDs, lam_con, p_con))
print("=============================")
# print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Non-cooperative:", Dmax_non)
print("Dmax Contract:", Dmax_con)

Nash Product SWM: 33.87869366086002
Nash Product ADMM: 33.86067989729551
Nash Product Proportional: nan
Nash Product Uniform: nan
Nash Product Non-cooperative: -86.52383586523801
Nash Product Contract: -inf
SW SWM: 88.33384951949053
SW ADMM: 88.2841377290702
SW Proportional: -49.979860581700706
SW Uniform: -50.85816304968958
SW Non-cooperative: 84.1729598354875
SW Contract: 97.0
Dmax SWM: 0.4765305235523106
Dmax ADMM: 0.4766778672211501
Dmax Proportional: 0.9779738123526548
Dmax Uniform: 0.9781732634978643
Dmax Non-cooperative: 0.749216419875129
Dmax Contract: 0


c:\Users\a1831\Desktop\NBS_project\src\model.py:11: RuntimeWarning: invalid value encountered in log
  n0 = ESP.omega_0 * np.log(ESP.Q(Dmax)-np.sum(r))
c:\Users\a1831\Desktop\NBS_project\src\model.py:12: RuntimeWarning: invalid value encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
c:\Users\a1831\Desktop\NBS_project\src\model.py:12: RuntimeWarning: divide by zero encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
